순환 신경망 모델에서 주로 사용하는 LSTM 레이어에 대해서 알아보겠습니다. 시퀀스 데이터를 다루는 레이어라 설정 파라미터에 따라 다양하게 모델을 구성할 수 있습니다. 그만큼 헷갈리는 부분도 있지만 쉬운 예제부터 차근히 살펴보겠습니다.

---

### 긴 시퀀스를 기억할 수 있는 LSTM (Long Short-Term Memory units)  레이어

케라스에서 제공하는 순환 신경망 레이어는 SimpleRNN, GRU, LSTM이 있으나 주로 사용하는 LSTM에 대해서 알아보겠습니다. 이 LSTM은 아래와 같이 간단히 사용할 수 있습니다.

    LSTM(3, input_dim=1)

주요 인자는 다음과 같습니다.
* 첫번째 인자 : 출력 뉴런의 수 입니다.
* input_dim : 입력 뉴런의 수 입니다.

이는 앞서 살펴본 Dense 레이어 형태와 비슷합니다.

    Dense(3, input_dim=1)
    
Dense와 LSTM을 레고 블록으로 도식화 하면 다음과 같습니다. 왼쪽이 Dense이고, 오른쪽이 LSTM 입니다.

![img](http://tykimos.github.com/Keras/warehouse/2017-4-9-RNN_Layer_Talk_1.png)
    
사실 LSTM의 내부구조는 복잡하지만 간소화하여 외형만 표시한 것입니다. Dense 레이어와 비교한다면 히든 뉴런들이 밖으로 도출되어 있음을 보실 수 있습니다. 이러한 히든 뉴런이 재귀적으로 다시 입력으로 들어가게 되어 순환 신경망이 되는 것입니다. 간단한 예제로 하나 하나 살펴보겠습니다. 

---

### 시퀀스 데이터 준비

순환 신경망은 주로 자연어 처리에 많이 쓰이기 때문에 문장 학습 예제가 일반적이지만 본 강좌에서는 악보 학습을 해보겠습니다. 그 이유는 
- 음계가 문장보다 더 코드화 하기 쉽고, 
- 시계열 자료이며, 
- 나온 결과를 악보로 볼 수 있으며,
- 무엇보다 우리가 학습한 모델이 연주하는 곡을 들어볼 수 있기 때문입니다. 
일단 쉬운 악보인 '나비야'를 준비했습니다.

![img](http://tykimos.github.com/Keras/warehouse/2017-4-9-RNN_Layer_Talk_2.png)

음표 밑에 간단한 음표코드를 표시하였습니다. 알파벳은 음계를 나타내며, 숫자는 음의 길이를 나타냅니다.
- c(도), d(레), e(미), f(파), g(솔), a(라), b(시)
- 4(4분음표), 8(8분음표)

---

### 데이터셋 생성

먼저 두 음절만 살펴보겠습니다. 

* g8 e8 e4 | f8 d8 d4 | 

여기서 우리가 정의한 문제대로 4개 음표 입력으로 다음 출력 음표를 예측하려면, 아래와 같이 데이터셋을 구성합니다.

* g8 e8 e4 f8 d8 : 1~4번째 음표, 5번째 음표
* e8 e4 f8 d8 d4 : 2~5번째 음표, 6번째 음표

6개의 음표로는 위와 같이 2개의 샘플이 나옵니다. 각 샘플은 4개의 입력 데이터와 1개의 라벨값으로 구성되어 있습니다. 즉 1~4번째 열은 속성(feature)이고, 5번째 열은 클래스(class)를 나타냅니다. 이렇게 4개씩 구간을 보는 것을 윈도우 크기가 4라고 합니다. 그리고 문자와 숫자로 된 음표(코드)로는 모델 입출력으로 사용할 수 없기 때문에 각 코드를 숫자로 변환할 수 있는 사전을 하나 만들어봅니다. 첫번째 사전은 코드를 숫자로, 두번째 사전은 숫자를 코드로 만드는 코드입니다.

In [130]:
code2idx = {'c4':0, 'd4':1, 'e4':2, 'f4':3, 'g4':4, 'a4':5, 'b4':6,
            'c8':7, 'd8':8, 'e8':9, 'f8':10, 'g8':11, 'a8':12, 'b8':13}

idx2code = {0:'c4', 1:'d4', 2:'e4', 3:'f4', 4:'g4', 5:'a4', 6:'b4',
            7:'c8', 8:'d8', 9:'e8', 10:'f8', 11:'g8', 12:'a8', 13:'b8'}

이러한 사전을 이용해서 순차적인 음표를 우리가 지정한 윈도우 크기만큼 잘라 데이터셋을 생성하는 함수를 정의해보겠습니다.

In [133]:
import numpy as np

def seq2dataset(seq, window_size):
    dataset = []
    for i in range(len(seq)-window_size):
        subset = seq[i:(i+window_size+1)]
        dataset.append([code2idx[item] for item in subset])
    return np.array(dataset)

seq라는 변수에 "나비야" 곡 전체 음표를 저장한 다음, seq2dataset() 함수를 하여 dataset를 생성합니다. 데이터셋은 앞서 정의한 사전에 따라 숫자로 변환되어 생성됩니다.

In [134]:
seq = ['g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'd8', 'e8', 'f8', 'g8', 'g8', 'g4',
       'g8', 'e8', 'e8', 'e8', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4',
       'd8', 'd8', 'd8', 'd8', 'd8', 'e8', 'f4', 'e8', 'e8', 'e8', 'e8', 'e8', 'f8', 'g4',
       'g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4']

dataset = seq2dataset(seq, window_size = 4)

print(dataset.shape)
print(dataset)

(50, 5)
[[11  9  2 10  8]
 [ 9  2 10  8  1]
 [ 2 10  8  1  7]
 [10  8  1  7  8]
 [ 8  1  7  8  9]
 [ 1  7  8  9 10]
 [ 7  8  9 10 11]
 [ 8  9 10 11 11]
 [ 9 10 11 11  4]
 [10 11 11  4 11]
 [11 11  4 11  9]
 [11  4 11  9  9]
 [ 4 11  9  9  9]
 [11  9  9  9 10]
 [ 9  9  9 10  8]
 [ 9  9 10  8  1]
 [ 9 10  8  1  7]
 [10  8  1  7  9]
 [ 8  1  7  9 11]
 [ 1  7  9 11 11]
 [ 7  9 11 11  9]
 [ 9 11 11  9  9]
 [11 11  9  9  2]
 [11  9  9  2  8]
 [ 9  9  2  8  8]
 [ 9  2  8  8  8]
 [ 2  8  8  8  8]
 [ 8  8  8  8  8]
 [ 8  8  8  8  9]
 [ 8  8  8  9  3]
 [ 8  8  9  3  9]
 [ 8  9  3  9  9]
 [ 9  3  9  9  9]
 [ 3  9  9  9  9]
 [ 9  9  9  9  9]
 [ 9  9  9  9 10]
 [ 9  9  9 10  4]
 [ 9  9 10  4 11]
 [ 9 10  4 11  9]
 [10  4 11  9  2]
 [ 4 11  9  2 10]
 [11  9  2 10  8]
 [ 9  2 10  8  1]
 [ 2 10  8  1  7]
 [10  8  1  7  9]
 [ 8  1  7  9 11]
 [ 1  7  9 11 11]
 [ 7  9 11 11  9]
 [ 9 11 11  9  9]
 [11 11  9  9  2]]


---

### 학습 과정

"나비야"노래는 우리에게 너무나 익숙한 노래입니다. 만약 옆사람이 "나비야~ 나"까지만 불러도 나머지를 이어서 다 부를 수 있을 정도로 말이죠. 이렇게 첫 4개 음표를 입력하면 나머지를 연주할 수 있는 모델을 만드는 것이 목표입니다. 우리가 정의한 문제를 풀기 위해 먼저 모델을 학습시켜야 합니다. 학습 시키는 방식은 아래와 같습니다.

- 파란색 박스가 입력값이고, 빨간색 박스가 우리가 원하는 출력값입니다. 
- 1~4번째 음표를 데이터로 5번째 음표를 라벨값으로 학습을 시킵니다.
- 다음에는 2~5번째 음표를 데이터로 6번째 음표를 라벨값으로 학습을 시킵니다.
- 이후 한 음표씩 넘어가면서 노래 끝까지 학습시킵니다.

![img](http://tykimos.github.com/Keras/warehouse/2017-4-9-RNN_Layer_Talk_5.png)

---
### 예측 과정

예측은 두 가지 방법으로 해보겠습니다. `한 스텝 예측`과 `곡 전체 예측`입니다. 

#### 한 스텝 예측

한 스텝 예측이란 실제 음표 4개를 입력하여 다음 음표 1개를 예측하는 것을 반복하는 것입니다. 이 방법에서는 모델의 입력값으로는 항상 실제 음표가 들어갑니다.
- 모델에 t0, t1, t2, t3를 입력하면 y0 출력이 나옵니다. 
- 모델에 t1, t2, t3, t4를 입력하면 y1 출력이 나옵니다.
- 모델에 t2, t3, t4, t5를 입력하면 y2 출력이 나옵니다.
- 이 과정을 y49 출력까지 반복합니다. 

![img](http://tykimos.github.com/Keras/warehouse/2017-4-9-RNN_Layer_Talk_6.png)

#### 곡 전체 예측

곡 전체 예측이란 입력된 초가 4개 음표만을 입력으로 곡 전체를 예측하는 것입니다. 초반부가 지나면, 예측값만으로 모델에 입력되어 다음 예측값이 나오는 식입니다. 그야말로 "나비야~ 나"까지 알려주면 나머지까지 모두 연주를 하는 것이죠. 만약 중간에 틀린 부분이 생긴다면, 이후 음정, 박자는 모두 이상하게 될 가능성이 많습니다. 예측 오류가 누적되는 것이겠죠.

- 모델에 t0, t1, t2, t3를 입력하면 y0 출력이 나옵니다.
- 예측값인 y0를 t4라고 가정하고, 모델에 t1, t2, t3, t4을 입력하면 y1 출력이 나옵니다.
- 예측값인 y1을 t5라고 가정하고, 모델에 t2, t3, t4(예측값), t5(예측값)을 입력하면 y2 출력이 나옵니다.
- 이 과정을 y49 출력까지 반복합니다.

![img](http://tykimos.github.com/Keras/warehouse/2017-4-9-RNN_Layer_Talk_7.png)

---

### 다층 퍼셉트론 모델

앞서 생성한 데이터셋으로 먼저 다층 퍼셋트론 모델을 학습시켜보겠습니다. 

In [9]:
# 코드 사전 정의

code2idx = {'c4':0, 'd4':1, 'e4':2, 'f4':3, 'g4':4, 'a4':5, 'b4':6,
            'c8':7, 'd8':8, 'e8':9, 'f8':10, 'g8':11, 'a8':12, 'b8':13}

idx2code = {0:'c4', 1:'d4', 2:'e4', 3:'f4', 4:'g4', 5:'a4', 6:'b4',
            7:'c8', 8:'d8', 9:'e8', 10:'f8', 11:'g8', 12:'a8', 13:'b8'}

# 데이터셋 생성 함수

import numpy as np

def seq2dataset(seq, window_size):
    dataset = []
    for i in range(len(seq)-window_size):
        subset = seq[i:(i+window_size+1)]
        dataset.append([code2idx[item] for item in subset])
    return np.array(dataset)

# 시퀀스 데이터 정의

seq = ['g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'd8', 'e8', 'f8', 'g8', 'g8', 'g4',
       'g8', 'e8', 'e8', 'e8', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4',
       'd8', 'd8', 'd8', 'd8', 'd8', 'e8', 'f4', 'e8', 'e8', 'e8', 'e8', 'e8', 'f8', 'g4',
       'g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4']

# 데이터셋 생성

dataset = seq2dataset(seq, window_size = 4)

print(dataset.shape)
print(dataset)

from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils

# 랜덤시드 고정시키기
np.random.seed(5)

# 입력(X)과 출력(Y) 변수로 분리하기
train_X = dataset[:,0:4]
train_Y = dataset[:,4]

max_idx_value = 13

# 입력값 정규화 시키기
train_X = train_X / float(max_idx_value)

# 라벨값에 대한 one-hot 인코딩 수행
train_Y = np_utils.to_categorical(train_Y)

one_hot_vec_size = train_Y.shape[1]

print("one hot encoding vector size is ", one_hot_vec_size)

# 모델 구성하기
model = Sequential()
model.add(Dense(128, input_dim=4, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(one_hot_vec_size, activation='softmax'))

# 모델 엮기
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 학습시키기
model.fit(train_X, train_Y, epochs=2000, batch_size=10, verbose=2)

# 모델 평가하기
scores = model.evaluate(train_X, train_Y)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 예측하기

pred_count = 50 # 최대 예측 개수 정의

# 한 스텝 예측

seq_out = ['g8', 'e8', 'e4', 'f8']
pred_out = model.predict(train_X)

for i in range(pred_count):
    idx = np.argmax(pred_out[i]) # one-hot 인코딩을 인덱스 값으로 변환
    seq_out.append(idx2code[idx]) # seq_out는 최종 악보이므로 인덱스 값을 코드로 변환하여 저장
    
print("one step prediction : ", seq_out)

# 곡 전체 예측

seq_in = ['g8', 'e8', 'e4', 'f8']
seq_out = seq_in
seq_in = [code2idx[it] for it in seq_in] # 코드를 인덱스값으로 변환

for i in range(pred_count):
    sample_in = np.array(seq_in)
    sample_in = np.reshape(sample_in, (1, 4)) # batch_size, feature
    pred_out = model.predict(sample_in)
    idx = np.argmax(pred_out)
    seq_out.append(idx2code[idx])
    seq_in.append(idx)
    seq_in.pop(0)

print("full song prediction : ", seq_out)

(50, 5)
[[11  9  2 10  8]
 [ 9  2 10  8  1]
 [ 2 10  8  1  7]
 [10  8  1  7  8]
 [ 8  1  7  8  9]
 [ 1  7  8  9 10]
 [ 7  8  9 10 11]
 [ 8  9 10 11 11]
 [ 9 10 11 11  4]
 [10 11 11  4 11]
 [11 11  4 11  9]
 [11  4 11  9  9]
 [ 4 11  9  9  9]
 [11  9  9  9 10]
 [ 9  9  9 10  8]
 [ 9  9 10  8  1]
 [ 9 10  8  1  7]
 [10  8  1  7  9]
 [ 8  1  7  9 11]
 [ 1  7  9 11 11]
 [ 7  9 11 11  9]
 [ 9 11 11  9  9]
 [11 11  9  9  2]
 [11  9  9  2  8]
 [ 9  9  2  8  8]
 [ 9  2  8  8  8]
 [ 2  8  8  8  8]
 [ 8  8  8  8  8]
 [ 8  8  8  8  9]
 [ 8  8  8  9  3]
 [ 8  8  9  3  9]
 [ 8  9  3  9  9]
 [ 9  3  9  9  9]
 [ 3  9  9  9  9]
 [ 9  9  9  9  9]
 [ 9  9  9  9 10]
 [ 9  9  9 10  4]
 [ 9  9 10  4 11]
 [ 9 10  4 11  9]
 [10  4 11  9  2]
 [ 4 11  9  2 10]
 [11  9  2 10  8]
 [ 9  2 10  8  1]
 [ 2 10  8  1  7]
 [10  8  1  7  9]
 [ 8  1  7  9 11]
 [ 1  7  9 11 11]
 [ 7  9 11 11  9]
 [ 9 11 11  9  9]
 [11 11  9  9  2]]
('one hot encoding vector size is ', 12)
Epoch 1/2000
0s - loss: 2.4535 - acc: 0.2000
Epoch

한 스텝 예측 결과를 악보로 그려보았고, 이 중 틀린 부분을 빨간색 박스로 표시해봤습니다. 총 50개 예측 중 4개가 틀려서 92%의 정확도가 나왔습니다.

![img](http://tykimos.github.com/Keras/warehouse/2017-4-9-RNN_Layer_Talk_butterfly_MLP_one_step_prediction.png)

곡 전체 예측 결과를 악보로 표시해봤습니다. 중간에 틀릭 부분이 생기면 곡 전체를 예측하는 데 있어서는 그리 좋은 성능이 나오지 않습니다. MLP에서는 입력 4개 음표에 대해서 선후 관계를 모르기 때문에 단순히 다른 입력이라고 보고 다른 결과를 도출합니다.

![img](http://tykimos.github.com/Keras/warehouse/2017-4-9-RNN_Layer_Talk_butterfly_MLP_full_song_prediction.png)

---

### LSTM 첫번째 모델

이번에는 간단한 LSTM 모델로 먼저 테스트를 해보겠습니다. 모델 구성은 다음과 같이 하였습니다.
- 128 뉴런을 가진 LSTM 레이어 1개와 Dense 레이어로 구성
- 입력은 샘플이 50개, 타임스텝이 4개, 속성이 1개로 구성
- 상태유지 안함 (stateless)

케라스에서는 아래와 같이 LSTM을 구성할 수 있습니다.

In [14]:
model = Sequential()
model.add(LSTM(128, input_shape = (4, 1)))
model.add(Dense(one_hot_vec_size, activation='softmax'))

LSTM을 제대로 활용하기 위해서는 `상태유지 모드`, `배치사이즈`, `타임스텝`, `속성`에 대한 개념에 이해가 필요합니다. 본 절에서는 `타임스텝`에 대해서 먼저 알아보겠습니다. `타임스텝`이란 하나의 샘플에 포함된 시퀀스 개수입니다. 현재 문제에서는 매 샘플마다 4개의 값을 입력하므로 타임스텝이 4개로 지정할 수 있습니다. 즉 윈도우 크기와 동일하게 타임스텝으로 설정하면 됩니다. `속성`에 대해서는 나중에 알아보겠지만, 입력되는 음표 1개당 하나의 인덱스 값을 입력하므로 속성이 1개입니다. 나중에 이 `속성`의 개수를 다르게 해서 테스트 해보겠습니다. 

LSTM에 입력할 데이터셋은 샘플 수, 타임스텝 수, 속성 수 형식으로 맞추어야 합니다. 따라서 앞서 구성한 train_X를 아래와 같이 형식을 변환합니다.

In [ ]:
train_X = np.reshape(train_X, (50, 4, 1))

전체 소스는 다음과 같습니다.

In [13]:
# 코드 사전 정의

code2idx = {'c4':0, 'd4':1, 'e4':2, 'f4':3, 'g4':4, 'a4':5, 'b4':6,
            'c8':7, 'd8':8, 'e8':9, 'f8':10, 'g8':11, 'a8':12, 'b8':13}

idx2code = {0:'c4', 1:'d4', 2:'e4', 3:'f4', 4:'g4', 5:'a4', 6:'b4',
            7:'c8', 8:'d8', 9:'e8', 10:'f8', 11:'g8', 12:'a8', 13:'b8'}

# 데이터셋 생성 함수

import numpy as np

def seq2dataset(seq, window_size):
    dataset = []
    for i in range(len(seq)-window_size):
        subset = seq[i:(i+window_size+1)]
        dataset.append([code2idx[item] for item in subset])
    return np.array(dataset)

# 시퀀스 데이터 정의

seq = ['g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'd8', 'e8', 'f8', 'g8', 'g8', 'g4',
       'g8', 'e8', 'e8', 'e8', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4',
       'd8', 'd8', 'd8', 'd8', 'd8', 'e8', 'f4', 'e8', 'e8', 'e8', 'e8', 'e8', 'f8', 'g4',
       'g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4']

# 데이터셋 생성

dataset = seq2dataset(seq, window_size = 4)

print(dataset.shape)

from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils

# 랜덤시드 고정시키기
np.random.seed(5)

# 입력(X)과 출력(Y) 변수로 분리하기
train_X = dataset[:,0:4]
train_Y = dataset[:,4]

max_idx_value = 13

# 입력값 정규화 시키기
train_X = train_X / float(max_idx_value)

# 입력을 (샘플 수, 타입스텝, 특성 수)로 형태 변환
train_X = np.reshape(train_X, (50, 4, 1))

# 라벨값에 대한 one-hot 인코딩 수행
train_Y = np_utils.to_categorical(train_Y)

one_hot_vec_size = train_Y.shape[1]

print("one hot encoding vector size is ", one_hot_vec_size)

# 모델 구성하기
model = Sequential()
model.add(LSTM(128, input_shape = (4, 1)))
model.add(Dense(one_hot_vec_size, activation='softmax'))

# 모델 엮기
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 학습시키기
model.fit(train_X, train_Y, epochs=2000, batch_size=1, verbose=2)

# 모델 평가하기
scores = model.evaluate(train_X, train_Y)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 예측하기

pred_count = 50 # 최대 예측 개수 정의

# 한 스텝 예측

seq_out = ['g8', 'e8', 'e4', 'f8']
pred_out = model.predict(train_X)

for i in range(pred_count):
    idx = np.argmax(pred_out[i]) # one-hot 인코딩을 인덱스 값으로 변환
    seq_out.append(idx2code[idx]) # seq_out는 최종 악보이므로 인덱스 값을 코드로 변환하여 저장
    
print("one step prediction : ", seq_out)

# 곡 전체 예측

seq_in = ['g8', 'e8', 'e4', 'f8']
seq_out = seq_in
seq_in = [code2idx[it] for it in seq_in] # 코드를 인덱스값으로 변환

for i in range(pred_count):
    sample_in = np.array(seq_in)
    sample_in = np.reshape(sample_in, (1, 4, 1)) # 샘플 수, 타입스텝 수, 속성 수
    pred_out = model.predict(sample_in)
    idx = np.argmax(pred_out)
    seq_out.append(idx2code[idx])
    seq_in.append(idx)
    seq_in.pop(0)

print("full song prediction : ", seq_out)

(50, 5)
('one hot encoding vector size is ', 12)
Epoch 1/2000
0s - loss: 2.3882 - acc: 0.2800
Epoch 2/2000
0s - loss: 2.0798 - acc: 0.3400
Epoch 3/2000
0s - loss: 1.9695 - acc: 0.3400
Epoch 4/2000
0s - loss: 1.9558 - acc: 0.3400
Epoch 5/2000
0s - loss: 1.9454 - acc: 0.3400
Epoch 6/2000
0s - loss: 1.9433 - acc: 0.3400
Epoch 7/2000
0s - loss: 1.9402 - acc: 0.3400
Epoch 8/2000
0s - loss: 1.9267 - acc: 0.3400
Epoch 9/2000
0s - loss: 1.9120 - acc: 0.3400
Epoch 10/2000
0s - loss: 1.9264 - acc: 0.3400
Epoch 11/2000
0s - loss: 1.9198 - acc: 0.3400
Epoch 12/2000
0s - loss: 1.9184 - acc: 0.3400
Epoch 13/2000
0s - loss: 1.9159 - acc: 0.3400
Epoch 14/2000
0s - loss: 1.9369 - acc: 0.3400
Epoch 15/2000
0s - loss: 1.9266 - acc: 0.3400
Epoch 16/2000
0s - loss: 1.9070 - acc: 0.3400
Epoch 17/2000
0s - loss: 1.9086 - acc: 0.3400
Epoch 18/2000
0s - loss: 1.9073 - acc: 0.3400
Epoch 19/2000
0s - loss: 1.9110 - acc: 0.3400
Epoch 20/2000
0s - loss: 1.9100 - acc: 0.3400
Epoch 21/2000
0s - loss: 1.9042 - acc: 0

In [ ]:





num_epoch = 300

# 모델 학습시키기
for epoch_idx in range(num_epoch):
    print ('epochs : ' + str(epoch_idx) )
    model.fit(train_X, train_Y, epochs=1, batch_size=1, verbose=2, shuffle=False) # 50 is X.shape[0]
    model.reset_states()
    
# 모델 평가하기
scores = model.evaluate(train_X, train_Y, batch_size=1, verbose=1)
model.reset_states()

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 악보 생성하기

# 최대 예측 개수 정의
prediction_count = 50

# '나비야~나'까지 입력
seq_in = ['g8', 'e8', 'e4', 'f8']
seq_out = seq_in

# 입력 코드를 인덱스화 시킨 다음 정규화 처리
seq_in = [code2idx[item] / float(14) for item in seq_in]

for i in range(prediction_count):
    test_x = np.array(seq_in)
    test_x = np.reshape(test_x, (1, 4, 1)) # (batch_size, timesteps, data_dim)
    
    # 4개를 입력하여 one-hot 인코딩 된 결과 얻음
    prediction_onehot = model.predict(test_x)
    
    # one-hot 인코딩을 인덱스 값으로 변환
    prediction_index = np.argmax(prediction_onehot)
    
    # seq_out는 최종 악보이므로 인덱스 값을 코드로 변환하여 저장
    seq_out.append(idx2code[prediction_index])
    
    # 다음 예측을 위해 seq_in에 신규 항목을 추가하고, 기존 항목 제거
    seq_in.append(prediction_index / float(14))
    seq_in.pop(0)
    
print(seq_out)

In [6]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.utils import np_utils
import numpy as np

# 코드와 인덱스 매칭 테이블 정의
code2idx = {'c4':0, 'd4':1, 'e4':2, 'f4':3, 'g4':4, 'a4':5, 'b4':6,
            'c8':7, 'd8':8, 'e8':9, 'f8':10, 'g8':11, 'a8':12, 'b8':13}

idx2code = {0:'c4', 1:'d4', 2:'e4', 3:'f4', 4:'g4', 5:'a4', 6:'b4',
            7:'c8', 8:'d8', 9:'e8', 10:'f8', 11:'g8', 12:'a8', 13:'b8'}

# 시퀀스 데이터로부터 데이터셋 생성 함수
def seq2dataset(seq, window_size):
    dataset = []
    for i in range(len(seq)-window_size):
        subset = seq[i:(i+window_size+1)]
        dataset.append([code2idx[item] for item in subset])
    return np.array(dataset)

# 랜덤시드 고정시키기
np.random.seed(5)

# "나비야" 코드
seq = ['g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'd8', 'e8', 'f8', 'g8', 'g8', 'g4', 
       'g8', 'e8', 'e8', 'e8', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4',
       'd8', 'd8', 'd8', 'd8', 'd8', 'e8', 'f4', 'e8', 'e8', 'e8', 'e8', 'e8', 'f8', 'g4',
       'g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4']

# 윈도우 크기가 4로 데이터셋 생성
dataset = seq2dataset(seq, 4)

# 입력(X)과 출력(Y) 변수로 분리하기
train_X = dataset[:,0:4]
train_Y = dataset[:,4]

max_idx_value = 13

# 입력에 대해서 정규화 함
train_X = train_X / float(max_idx_value)

# 입력을 (샘플 수, 타입스텝, 특성 수)로 형태 변환
train_X = np.reshape(train_X, (train_X.shape[0], train_X.shape[1], 1))

# 인덱스로부터 다중 클래스 라벨을 위한 one-hot 인코드로 변환
train_Y = np_utils.to_categorical(train_Y)

# 모델 정의
# batch_input_shape = (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(128, input_shape = (4, 1)))
model.add(Dense(train_Y.shape[1], activation='softmax'))

# 모델 엮기
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_X, train_Y, epochs=2000, batch_size=50, verbose=2, shuffle=False) # 50 is X.shape[0]
    
# 모델 평가하기
scores = model.evaluate(train_X, train_Y, batch_size=50, verbose=1)

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 악보 생성하기

# 최대 예측 개수 정의
prediction_count = 50

# '나비야~나'까지 입력
seq_in = ['g8', 'e8', 'e4', 'f8']
seq_out = seq_in

# 입력 코드를 인덱스화 시킨 다음 정규화 처리
seq_in = [code2idx[item] / float(max_idx_value) for item in seq_in]

for i in range(prediction_count):
    test_x = np.array(seq_in)
    test_x = np.reshape(test_x, (1, 4, 1)) # (batch_size, timesteps, data_dim)
    
    # 4개를 입력하여 one-hot 인코딩 된 결과 얻음
    prediction_onehot = model.predict(test_x)
    
    # one-hot 인코딩을 인덱스 값으로 변환
    prediction_index = np.argmax(prediction_onehot)
    
    # seq_out는 최종 악보이므로 인덱스 값을 코드로 변환하여 저장
    seq_out.append(idx2code[prediction_index])
    
    # 다음 예측을 위해 seq_in에 신규 항목을 추가하고, 기존 항목 제거
    seq_in.append(prediction_index / float(max_idx_value))
    seq_in.pop(0)
    
print(seq_out)

Epoch 1/2000
0s - loss: 2.4806 - acc: 0.0600
Epoch 2/2000
0s - loss: 2.4685 - acc: 0.0600
Epoch 3/2000
0s - loss: 2.4567 - acc: 0.3400
Epoch 4/2000
0s - loss: 2.4449 - acc: 0.3400
Epoch 5/2000
0s - loss: 2.4332 - acc: 0.3400
Epoch 6/2000
0s - loss: 2.4215 - acc: 0.3400
Epoch 7/2000
0s - loss: 2.4096 - acc: 0.3400
Epoch 8/2000
0s - loss: 2.3975 - acc: 0.3400
Epoch 9/2000
0s - loss: 2.3851 - acc: 0.3400
Epoch 10/2000
0s - loss: 2.3723 - acc: 0.3400
Epoch 11/2000
0s - loss: 2.3590 - acc: 0.3400
Epoch 12/2000
0s - loss: 2.3452 - acc: 0.3400
Epoch 13/2000
0s - loss: 2.3307 - acc: 0.3400
Epoch 14/2000
0s - loss: 2.3155 - acc: 0.3400
Epoch 15/2000
0s - loss: 2.2995 - acc: 0.3400
Epoch 16/2000
0s - loss: 2.2826 - acc: 0.3400
Epoch 17/2000
0s - loss: 2.2648 - acc: 0.3400
Epoch 18/2000
0s - loss: 2.2460 - acc: 0.3400
Epoch 19/2000
0s - loss: 2.2261 - acc: 0.3400
Epoch 20/2000
0s - loss: 2.2053 - acc: 0.3400
Epoch 21/2000
0s - loss: 2.1835 - acc: 0.3400
Epoch 22/2000
0s - loss: 2.1609 - acc: 0.34

케라스 버전

In [116]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.utils import np_utils
import numpy as np

# 코드와 인덱스 매칭 테이블 정의
code2idx = {'c4':0, 'd4':1, 'e4':2, 'f4':3, 'g4':4, 'a4':5, 'b4':6,
            'c8':7, 'd8':8, 'e8':9, 'f8':10, 'g8':11, 'a8':12, 'b8':13}

idx2code = {0:'c4', 1:'d4', 2:'e4', 3:'f4', 4:'g4', 5:'a4', 6:'b4',
            7:'c8', 8:'d8', 9:'e8', 10:'f8', 11:'g8', 12:'a8', 13:'b8'}

# 시퀀스 데이터로부터 데이터셋 생성 함수
def seq2dataset(seq, window_size):
    dataset = []
    for i in range(len(seq)-window_size):
        subset = seq[i:(i+window_size+1)]
        dataset.append([code2int[item] for item in subset])
    return np.array(dataset)

# 랜덤시드 고정시키기
np.random.seed(5)

# "나비야" 코드
seq = ['g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'd8', 'e8', 'f8', 'g8', 'g8', 'g4', 
       'g8', 'e8', 'e8', 'e8', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4',
       'd8', 'd8', 'd8', 'd8', 'd8', 'e8', 'f4', 'e8', 'e8', 'e8', 'e8', 'e8', 'f8', 'g4',
       'g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4']

# 윈도우 크기가 4로 데이터셋 생성
dataset = seq2dataset(seq, 4)

# 입력(X)과 출력(Y) 변수로 분리하기
train_X = dataset[:,0:4]
train_Y = dataset[:,4]

max_idx_value = 13

# 입력에 대해서 정규화 함
train_X = train_X / float(max_idx_value)

# 입력을 (샘플 수, 타입스텝, 특성 수)로 형태 변환
train_X = np.reshape(train_X, (train_X.shape[0], train_X.shape[1], 1))

# 인덱스로부터 다중 클래스 라벨을 위한 one-hot 인코드로 변환
train_Y = np_utils.to_categorical(train_Y)

# 모델 정의
# batch_input_shape = (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(128, batch_input_shape = (1, 4, 1), stateful=True))
model.add(Dense(train_Y.shape[1], activation='softmax'))

# 모델 엮기
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

num_epoch = 300

# 모델 학습시키기
for epoch_idx in range(num_epoch):
    print ('epochs : ' + str(epoch_idx) )
    model.fit(train_X, train_Y, epochs=1, batch_size=1, verbose=2, shuffle=False) # 50 is X.shape[0]
    model.reset_states()
    
# 모델 평가하기
scores = model.evaluate(train_X, train_Y, batch_size=1, verbose=1)
model.reset_states()

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 악보 생성하기

# 최대 예측 개수 정의
prediction_count = 50

# '나비야~나'까지 입력
seq_in = ['g8', 'e8', 'e4', 'f8']
seq_out = seq_in

# 입력 코드를 인덱스화 시킨 다음 정규화 처리
seq_in = [code2idx[item] / float(max_idx_value) for item in seq_in]

for i in range(prediction_count):
    test_x = np.array(seq_in)
    test_x = np.reshape(test_x, (1, 4, 1)) # (batch_size, timesteps, data_dim)
    
    # 4개를 입력하여 one-hot 인코딩 된 결과 얻음
    prediction_onehot = model.predict(test_x)
    
    # one-hot 인코딩을 인덱스 값으로 변환
    prediction_index = np.argmax(prediction_onehot)
    
    # seq_out는 최종 악보이므로 인덱스 값을 코드로 변환하여 저장
    seq_out.append(idx2code[prediction_index])
    
    # 다음 예측을 위해 seq_in에 신규 항목을 추가하고, 기존 항목 제거
    seq_in.append(prediction_index / float(max_idx_value))
    seq_in.pop(0)
    
print(seq_out)

epochs : 0
Epoch 1/1
2s - loss: 2.3483 - acc: 0.1400
epochs : 1
Epoch 1/1
0s - loss: 2.0417 - acc: 0.3400
epochs : 2
Epoch 1/1
0s - loss: 1.9635 - acc: 0.3400
epochs : 3
Epoch 1/1
0s - loss: 1.9469 - acc: 0.3400
epochs : 4
Epoch 1/1
0s - loss: 1.9369 - acc: 0.3400
epochs : 5
Epoch 1/1
0s - loss: 1.9301 - acc: 0.3400
epochs : 6
Epoch 1/1
0s - loss: 1.9248 - acc: 0.3600
epochs : 7
Epoch 1/1
0s - loss: 1.9205 - acc: 0.3600
epochs : 8
Epoch 1/1
0s - loss: 1.9168 - acc: 0.3600
epochs : 9
Epoch 1/1
0s - loss: 1.9134 - acc: 0.3600
epochs : 10
Epoch 1/1
0s - loss: 1.9102 - acc: 0.3600
epochs : 11
Epoch 1/1
0s - loss: 1.9070 - acc: 0.3600
epochs : 12
Epoch 1/1
0s - loss: 1.9036 - acc: 0.3600
epochs : 13
Epoch 1/1
0s - loss: 1.8997 - acc: 0.3600
epochs : 14
Epoch 1/1
0s - loss: 1.8954 - acc: 0.3600
epochs : 15
Epoch 1/1
0s - loss: 1.9070 - acc: 0.3600
epochs : 16
Epoch 1/1
0s - loss: 1.8938 - acc: 0.3600
epochs : 17
Epoch 1/1
0s - loss: 1.8822 - acc: 0.3600
epochs : 18
Epoch 1/1
0s - loss: 2.062

['g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'd8', 'e8', 'f8', 'g8', 'g8', 'g4', 'g8', 'e8', 'e8', 'e8', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4', 'd8', 'd8', 'd8', 'd8', 'd8', 'e8', 'f4', 'e8', 'e8', 'e8', 'e8', 'e8', 'f8', 'g4', 'g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4']


In [109]:
seq_in

[0.7857142857142857,
 0.6428571428571429,
 0.14285714285714285,
 0.7142857142857143]

In [ ]:





# as the first layer in a Sequential model
model = Sequential()
model.add(LSTM(32, input_shape=(10, 64)`))
# now model.output_shape == (None, 32)
# note: `None` is the batch dimension.

# for subsequent layers, no need to specify the input size:
model.add(LSTM(16))

# to stack recurrent layers, you must use return_sequences=True
# on any recurrent layer that feeds into another recurrent layer.
# note that you only need to specify the input size on the first layer.
model = Sequential()
model.add(LSTM(64, input_dim=64, input_length=10, return_sequences=True))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(10))

---

### 결론

본 강좌를 통해 컨볼루션 모델에서 사용되는 주요 레이어에 대해서 알아보았고, 레이어를 조합하여 컨볼루션 모델을 만들어봤습니다. 다음 강좌에서는 이 모델을 이용하여 실제로 데이터셋을 학습시킨 후 분류가 제대로 되는 지 알아보겠습니다.

---

### 같이 보기

* [강좌 목차](https://tykimos.github.io/Keras/lecture/)
* 이전 : [딥러닝 이야기/다층 퍼셉트론 모델 만들어보기](https://tykimos.github.io/Keras/2017/02/04/MLP_Getting_Started/)
* 다음 : [딥러닝 이야기/컨볼루션 신경망 모델 만들어보기](https://tykimos.github.io/Keras/2017/03/08/CNN_Getting_Started/)

In [ ]:
\paper { 
  indent = 0\mm
}

\header{
  title = "나 비 야"
}

melody = \relative c'' {
\clef treble
\key c \major
\autoBeamOff
\time 2/4
g8 e8 e4 
f8 d8 d4 
c8 d8 e8 f8 
g8 g8 g4
\break
g8 e8 e8 e8 
f8 d8 d4
c8 e8 g8 g8 
e8 e8 e4
\break
d8 d8 d8 d8 
d8 e8 f4 
e8 e8 e8 e8 
e8 f8 g4
\break
g8 e8 e4 
f8 d8 d4 
c8 e8 g8 g8 
e8 e8 e4
}

\addlyrics {
#"g8" #"e8" #"e4"
#"f8" #"d8" #"d4"
#"c8" #"d8" #"e8" #"f8"
#"g8" #"g8" #"g4"
#"g8" #"e8" #"e8" #"e8" 
#"f8" #"d8" #"d4"
#"c8" #"e8" #"g8" #"g8"
#"e8" #"e8" #"e4"
#"d8" #"d8" #"d8" #"d8"
#"d8" #"e8" #"f4"
#"e8" #"e8" #"e8" #"e8"
#"e8" #"f8" #"g4"
#"g8" #"e8" #"e4" 
#"f8" #"d8" #"d4" 
#"c8" #"e8" #"g8" #"g8"
#"e8" #"e8" #"e4"
}

\score {
  \new Staff \melody
  \layout { }
  \midi { }
}

\version "2.18.2"  % necessary for upgrading to future LilyPond versions.
